#### gdal命令行工具
通过notebook演示,需在命令前加'!'.

影像裁剪: gdal_translate   
1. 重采样（指定分辨率）   
gdal_translate -co COMPRESS=LZW -tr 100 100 -r average $path_in $path_out    
2. 重采样（指定输出影像尺寸）   
gdal_translate -co COMPRESS=LZW -outsize 100 100 -r average $path_in $path_out      
3. 裁剪（基于空间范围, extent: 'left up right down', 例如: extent='72 38 84 34'）  
gdal_translate -co COMPRESS=LZW -projwin $extent $path_in $path_out     
(参考：https://gdal.org/programs/gdal_translate.html)    



In [1]:
path_in = 'data/Section-2/s2_kunming_chenggong_4bands_10m.tif'   
path_resam1 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_resam25m.tif'   
path_resam2 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_resam100x100.tif'   
path_subs = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs2.tif'   
extent = '280200 2750700 281400 2748300'      
# !gdal_translate -co COMPRESS=LZW -tr 25 25 -r average $path_in $path_resam1   
# !gdal_translate -co COMPRESS=LZW -outsize 100 100 -r average $path_in $path_resam2   
!gdal_translate -co COMPRESS=LZW -projwin $extent $path_in $path_subs  



Input file size is 2996, 3224
0...10...20...30...40...50...60...70...80...90...100 - done.


gdalwarp    
1. 坐标系转换(地理坐标系->投影坐标系)   
gdalwarp -co COMPRESS=LZW -overwrite -s_srs EPSG:4326 -t_srs EPSG:32644 -r bilinear input.tif output.tif     
1. 裁剪(基于矢量数据)    
gdalwarp -co COMPRESS=LZW -cutline $path_shp -crop_to_cutline $path_input $path_output     
参考：https://gdal.org/programs/gdalwarp.html


In [20]:
path_in = 'data/Section-2/s2_kunming_chenggong_4bands_10m.tif'      ### 投影: utm48->wgs84
# path_out = 'data/Section-5/s2_kunming_chenggong_4bands_10m_wgs84.tif'
# !gdalwarp -co COMPRESS=LZW -overwrite -s_srs EPSG:32648 -t_srs EPSG:4326 -r bilinear $path_in $path_out  

## 矢量裁剪
path_vec = 'data/Section-5/subs_1.gpkg'
path_subs = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs1_.tif'
!gdalwarp -co COMPRESS=LZW -cutline $path_vec -crop_to_cutline $path_in $path_subs 



Creating output file that is 533P x 363L.
Processing data/Section-2/s2_kunming_chenggong_4bands_10m.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.


影像拼接：gdal_merge（!!!低版本gdal可能存在问题）    
1. 影像拼接（-init:拼接影像初始化值。-n:拼接过程忽略值）  
gdal_merge.py -co COMPRESS=LZW -init 0 -o $path_out $path_in_1 $path_in_2 $path_in_3   
(参考：https://gdal.org/programs/gdal_merge.html)


In [18]:
# path_in_1 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs1.tif'
# path_in_2 = 'data/Section-5/s2_kunming_chenggong_4bands_10m_subs2.tif'
# path_out = 'data/Section-5/s2_kunming_chenggong_4bands_10m_mosaic.tif'
# !gdal_merge.py -co COMPRESS=LZW -o $path_out $path_in_1 $path_in_2


'gdal_merge' �����ڲ����ⲿ���Ҳ���ǿ����еĳ���
���������ļ���


gdal_polygonize.py   
1. 遥感影像矢量化（无法对特定DN值的像素进行矢量化。）    
gdal_polygonize.py $path_img $path_img2polygon     
参考：https://gdal.org/programs/gdal_polygonize.html）   



In [4]:
path_img = 'data/Section-5/s2_chenggong_mndwi_thre01.tif'
path_vec = 'data/Section-5/s2_chenggong_mndwi_thre01_poly.gpkg'
!gdal_polygonize.py $path_img $path_vec


0...10...20...30...40...50...60...70...80...90...100 - done.


C:\Users\HP\miniconda3\envs\venv\Scripts\gdal_polygonize.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  __import__('pkg_resources').run_script('GDAL==3.6.2', 'gdal_polygonize.py')


gdal_rasterize   
1. 矢量数据栅格化（-b:波段次序，1代表一个波段，-a：像元值对应的矢量数据属性名）     
gdal_rasterize -ts 1000 1000 -burn 1 $path_vec $path_raster   
参考：https://gdal.org/programs/gdal_rasterize.html  


In [7]:
path_vec = 'data/Section-5/s2_chenggong_mndwi_thre01_poly.gpkg'
path_raster = 'data/Section-5/s2_chenggong_mndwi_thre01_poly2raster.tif'
!gdal_rasterize -ts 1000 1000 -burn 10 $path_vec $path_raster 


0...10...20...30...40...50...60...70...80...90...100 - done.
